In [9]:
from langchain_ollama import ChatOllama

# Initialize the Ollama model
llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0  # Ensures deterministic JSON output
)


In [10]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

# Define a structured model for a joke
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


In [12]:
# Initialize the JSON output parser with the Pydantic schema
parser = JsonOutputParser(pydantic_object=Joke)

# Define an explicit prompt to enforce JSON output

prompt = PromptTemplate(
    template="""
You are an AI assistant that **must** return valid JSON output.
Ensure the response follows this JSON schema:

{format_instructions}

Now, answer the following user query in **strict JSON format**:
{query}
""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [13]:
# Query for a joke
joke_query = "Tell me a joke."

# Combine the prompt, model, and parser into a processing chain
chain = prompt | llm | parser

# Get the parsed JSON response
parsed_joke = chain.invoke({"query": joke_query})

# Print the structured output
print(parsed_joke)


{'properties': {'setup': {'description': 'What do you call a fake noodle?', 'title': 'Setup', 'type': 'string'}, 'punchline': {'description': 'An impasta!', 'title': 'Punchline', 'type': 'string'}}, 'required': ['setup', 'punchline']}


In [14]:
# Stream parsed JSON output
for chunk in chain.stream({"query": joke_query}):
    print(chunk)


{}
{'properties': {}}
{'properties': {'setup': {}}}
{'properties': {'setup': {'description': ''}}}
{'properties': {'setup': {'description': 'What'}}}
{'properties': {'setup': {'description': 'What do'}}}
{'properties': {'setup': {'description': 'What do you'}}}
{'properties': {'setup': {'description': 'What do you call'}}}
{'properties': {'setup': {'description': 'What do you call a'}}}
{'properties': {'setup': {'description': 'What do you call a fake'}}}
{'properties': {'setup': {'description': 'What do you call a fake nood'}}}
{'properties': {'setup': {'description': 'What do you call a fake noodle'}}}
{'properties': {'setup': {'description': 'What do you call a fake noodle?'}}}
{'properties': {'setup': {'description': 'What do you call a fake noodle?', 'title': ''}}}
{'properties': {'setup': {'description': 'What do you call a fake noodle?', 'title': 'Setup'}}}
{'properties': {'setup': {'description': 'What do you call a fake noodle?', 'title': 'Setup', 'type': ''}}}
{'properties': 

In [15]:
# Initialize JsonOutputParser without a specific schema
parser = JsonOutputParser()

# Define a prompt
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [16]:
prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, template='Answer the user query.\n{format_instructions}\n{query}\n')

In [17]:
# Combine the prompt, model, and parser
chain = prompt | llm | parser

# Get the response
parsed_response = chain.invoke({"query": joke_query})

# Print the JSON output
print(parsed_response)


{'joke': {'setup': 'Why did the scarecrow win an award?', 'punchline': 'Because he was outstanding in his field!'}}
